Empezaremos analizando en total contamos con 6 tablas, se considerara como la tabla "sale" como tabla de hecho

In [8]:
SELECT top 5 * FROM [dbo].[Countries]
SELECT top 5 * FROM [dbo].[Customer]
SELECT top 5 * FROM [dbo].[orden_status]
SELECT top 5 * FROM [dbo].[Products]
SELECT top 5 * FROM [dbo].[Region]
SELECT top 5 * FROM [dbo].[Sales]

(5 rows affected)

(5 rows affected)

(5 rows affected)

(5 rows affected)

(5 rows affected)

...

id_customer,date_creation,customer_name,id_region,id_country
1,2023-10-31,Connor Johnson,3,10
2,2023-11-20,Olivia Smith,1,7
3,2023-11-03,Liam Brown,4,3
4,2023-10-18,Emma Davis,2,2
5,2023-11-20,Noah Miller,1,5


id_status,description
1001,Created
1002,Processed
1003,Fulfilled
1004,Returned
1005,Cancelled


id_product,product_name,category,unit_price
1,Air Jordan 1,Jordan,175.000000
2,Air Force 1,Air Force,105.000000
3,Dunk Low,Dunk,125.000000
4,Air Max 90,Air Max,140.000000
5,Blazer Mid,Blazer,95.000000


id_region,continent
1,America
2,Europe
3,Asia
4,Africa
5,Australia


id_sale,date_sale,id_customer,id_product,quantity,id_status
1,2023-12-15,42,13,12,1001
2,2023-12-05,29,8,11,1002
3,2023-10-29,5,23,8,1003
4,2023-11-20,30,10,9,1004
5,2023-12-17,20,1,13,1005


Se procede a sacar los numeros de caracteres para las columnas de texto

In [10]:

SELECT MAX(LEN(country)) FROM [dbo].[Countries]


SELECT MAX(LEN(customer_name)) FROM [dbo].[Customer]

SELECT MAX(LEN(description)) FROM [dbo].[orden_status]

SELECT MAX(LEN(category)) FROM [dbo].[Products]
SELECT MAX(LEN(product_name)) FROM [dbo].[Products]

SELECT MAX(LEN(continent)) FROM [dbo].[Region]

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

...

(No column name)
16


(No column name)
9


(No column name)
13


(No column name)
32


(No column name)
9


Analisando relacion de datos por un usuario aleatoreo

En este analisis procedemos a ver que el id\_region y id\_continent no tienen congruencia, ya que al hacer el join de las tablas no trae congruencia de de los paises con continentes.

\- Se considerara id\_region como la region donde se creo el usuario

\- Se considera id\_country como el pais donde reside el cliente junto con su continente

In [196]:
SELECT * FROM Customer WHERE customer_name = 'Olivia Smith'
SELECT * FROM Countries
SELECT * FROM Region

(1 row affected)

(10 rows affected)

(5 rows affected)

Total execution time: 00:00:00.441

id_customer,date_creation,customer_name,id_region,id_country
2,2023-11-20,Olivia Smith,1,7


id_country,country,id_region
1,Mexico,1
2,United Kingdom,2
3,Brasil,1
4,Japan,3
5,Egypt,4
6,Australia,5
7,New Zeland,5
8,Morocco,4
9,South Korea,3
10,Germany,2


id_region,continent
1,America
2,Europe
3,Asia
4,Africa
5,Australia


Se procede a crear una vista para simplificar el analisis de datos

In [200]:
CREATE VIEW VIZ_SALES 
AS
SELECT 
       SL.[id_sale]
      ,SL.[date_sale]
      ,CS.[date_creation]
      ,CS.[customer_name]
      ,RG.[continent] AS continent_created
      ,RGV.continent 
      ,CTS.[country] 
      ,PD.[product_name]
      ,PD.[category]
      ,OST.[description]
      ,PD.[unit_price]
      ,SL.[quantity]
      
FROM [dbo].[Sales] AS SL

LEFT JOIN Customer AS CS ON SL.id_customer = CS.id_customer
LEFT JOIN Region AS RG ON CS.id_region = RG.id_region

LEFT JOIN Countries AS CTS ON CS.id_country  = CTS.id_country
LEFT JOIN Region AS RGV ON CTS.id_region = RGV.id_region
LEFT JOIN Products AS PD ON SL.id_product =  PD.id_product 
LEFT JOIN orden_status AS OST ON SL.id_status = OST.id_status



Commands completed successfully.

Total execution time: 00:00:00.049

Validar la view que se creo

In [201]:
SELECT TOP 5 * FROM VIZ_SALES

(5 rows affected)

Total execution time: 00:00:00.049

id_sale,date_sale,date_creation,customer_name,continent_created,continent,country,product_name,category,description,unit_price,quantity
1,2023-12-15,2023-11-18,Zoey Campbell,Europe,America,Brasil,Free Run,Free,Created,115.000000,12
2,2023-12-05,2023-11-29,Emily Parker,Asia,Asia,Japan,React Element,React,Processed,150.000000,11
3,2023-10-29,2023-11-20,Noah Miller,America,Africa,Egypt,Air Jordan 3,Jordan,Fulfilled,215.000000,8
4,2023-11-20,2023-11-12,Jack Evans,Australia,Europe,Germany,SB Janoski,SB,Returned,95.000000,9
5,2023-12-17,2023-10-31,Madison King,America,Asia,Japan,Air Jordan 1,Jordan,Cancelled,175.000000,13


¿Cuál es la distribución geográfica de las ventas de productos Nike durante el último trimestre del año <span style="color: #09885a;">2023</span>?

In [202]:
SELECT 
    country
    ,SUM(unit_price) * SUM(quantity) AS Ventas
FROM VIZ_SALES 
GROUP BY 
    country


(9 rows affected)

Total execution time: 00:00:00.050

country,Ventas
Australia,877200.000000
Brasil,1829610.000000
Egypt,549600.000000
Germany,988900.000000
Japan,860720.000000
Mexico,1362270.000000
New Zeland,6649810.000000
South Korea,1833150.000000
United Kingdom,403000.000000


¿Cuál es el producto más vendido por país y por mes en el 2023?

In [203]:
WITH TGNR AS (
    SELECT 
        MONTH(date_sale) AS [Numero mes],
        country,
        product_name,
        SUM(quantity) AS [Unidades vendidas],
        SUM(unit_price * quantity) AS VentasTotales,
        ROW_NUMBER() OVER (PARTITION BY MONTH(date_sale),country ORDER BY SUM(unit_price * quantity) DESC) AS Rank
    FROM VIZ_SALES
    GROUP BY 
        product_name,
        MONTH(date_sale),
        country
)
SELECT 
    [Numero mes],
    country,
    product_name,
    VentasTotales AS Ventas,
    [Unidades vendidas]
FROM TGNR
WHERE Rank = 1
ORDER BY [Numero mes]


(27 rows affected)

Total execution time: 00:00:00.054

Numero mes,country,product_name,Ventas,Unidades vendidas
10,Australia,Air Max 95,4095.000000,21
10,Brasil,Air Jordan 3,3225.000000,15
10,Egypt,Nike x Sacai LDWaffle,2090.000000,11
10,Germany,Air Jordan 4,4085.000000,19
10,Japan,Air Max 90,2100.000000,15
10,Mexico,Air Max Plus,3500.000000,20
10,New Zeland,Travis Scott x Nike Air Jordan 1,2750.000000,10
10,South Korea,Air Max 95,5850.000000,30
10,United Kingdom,Air Max Plus,2275.000000,13
11,Australia,Air Jordan 3,6020.000000,28


¿Cuál es el cliente con más compras por país y por mes en el 2023?

In [204]:
WITH VentasPorMes AS (
    SELECT 
        MONTH(date_sale) AS [Numero mes],
        country,
        customer_name,
        SUM(unit_price * quantity) AS VentasTotales,
        ROW_NUMBER() OVER (PARTITION BY MONTH(date_sale), country ORDER BY SUM(unit_price * quantity) DESC) AS Rank
    FROM VIZ_SALES
    GROUP BY 
        country,
        MONTH(date_sale),
        customer_name
)
SELECT 
    [Numero mes],
    country,
    customer_name,
    VentasTotales AS Ventas
FROM VentasPorMes
WHERE Rank = 1
ORDER BY [Numero mes],country

(27 rows affected)

Total execution time: 00:00:00.061

Numero mes,country,customer_name,Ventas
10,Australia,Evelyn Hill,2975.000000
10,Brasil,Hailey Phillips,2580.000000
10,Egypt,Noah Miller,3770.000000
10,Germany,Jack Evans,5175.000000
10,Japan,Charlotte Wilson,4365.000000
10,Mexico,Ethan Carter,6535.000000
10,New Zeland,Levi Kelly,4860.000000
10,South Korea,Chloe Price,4900.000000
10,United Kingdom,Lucas Lee,3900.000000
11,Australia,Luke Baker,4585.000000


¿Cuál es el promedio de ventas por región o país durante el último trimestre del año 2023?

In [207]:
WITH TGEN AS (
    SELECT 
        continent,
        country,
        unit_price * quantity AS VentaTotal
    FROM VIZ_SALES
    WHERE YEAR(date_sale) = 2023
      AND MONTH(date_sale) >= 10 
      AND MONTH(date_sale) <= 12
)

SELECT
    continent,
    country,
    SUM(VentaTotal) AS VentalTotal,
    AVG(VentaTotal) AS PromedioVentas
FROM TGEN
GROUP BY 
    continent,
    country
ORDER BY 
    continent,
    country;



(9 rows affected)

Total execution time: 00:00:00.051

continent,country,VentalTotal,PromedioVentas
Africa,Egypt,23300.000000,1013.043478
America,Brasil,42860.000000,1020.476190
America,Mexico,41225.000000,1288.281250
Asia,Japan,31925.000000,1227.884615
Asia,South Korea,49545.000000,1303.815789
Australia,Australia,37540.000000,1564.166666
Australia,New Zeland,91700.000000,1291.549295
Europe,Germany,35065.000000,1298.703703
Europe,United Kingdom,24525.000000,1442.647058


¿Cuál es el producto más vendido durante el último trimestre del año 2023?

In [208]:
WITH TGEN AS (
    SELECT 
        product_name,
        SUM(unit_price * quantity) AS VentaTotal,
        ROW_NUMBER() OVER (ORDER BY SUM(unit_price * quantity) DESC) AS Rank
    FROM VIZ_SALES
    WHERE YEAR(date_sale) = 2023
      AND MONTH(date_sale) >= 10 -- Último trimestre: meses 10, 11 y 12
      AND MONTH(date_sale) <= 12
    GROUP BY 
        product_name
)

SELECT
    product_name,
    VentaTotal
FROM TGEN
WHERE Rank = 1
ORDER BY 
    VentaTotal DESC

(1 row affected)

Total execution time: 00:00:00.060

product_name,VentaTotal
Air Max 95,33735.000000


¿Cuál es el cliente con más compras durante el último trimestre del año 2023?

In [209]:
WITH TGEN AS (
    SELECT 
        customer_name,
        SUM(unit_price * quantity) AS VentaTotal,
        ROW_NUMBER() OVER (ORDER BY SUM(unit_price * quantity) DESC) AS Rank
    FROM VIZ_SALES
    WHERE YEAR(date_sale) = 2023
      AND MONTH(date_sale) >= 10 -- Último trimestre: meses 10, 11 y 12
      AND MONTH(date_sale) <= 12
    GROUP BY 
        customer_name
)

SELECT
    customer_name,
    VentaTotal
FROM TGEN
WHERE Rank = 1
ORDER BY 
    VentaTotal DESC

(1 row affected)

Total execution time: 00:00:00.161

customer_name,VentaTotal
Riley Martinez,14215.000000


¿Cuál es el promedio de cantidad de productos comprados por cliente?

In [212]:
    SELECT 
        customer_name,
        AVG(quantity) AS [%Promedio],
        SUM(quantity) AS [Total_unidades]
    FROM VIZ_SALES
    WHERE YEAR(date_sale) = 2023
      AND MONTH(date_sale) >= 10 -- Último trimestre: meses 10, 11 y 12
      AND MONTH(date_sale) <= 12
    GROUP BY 
        customer_name
    ORDER BY 
        AVG(quantity) DESC

(47 rows affected)

Total execution time: 00:00:00.065

customer_name,%Promedio,Total_unidades
Avery Nelson,15,15
Austin Cooper,10,64
Hannah Wright,10,72
Jack Evans,10,73
Luke Baker,10,60
Owen Turner,10,87
Peyton Morris,10,80
Riley Martinez,9,77
Scarlett Turner,9,18
Lucas Lee,9,56


¿Existe alguna tendencia de ventas en función del día de la semana?

In [213]:
WITH TGNR AS 
            (SELECT
                DiaSemana,
                ISNULL([10], 0) AS Octubre,
                ISNULL([11], 0) AS Noviembre,
                ISNULL([12], 0) AS Diciembre
            FROM (
                SELECT
                    DATEPART(dw, date_sale) AS DiaSemana,
                    MONTH(date_sale) AS Mes,
                    SUM(quantity * unit_price) AS TotalVenta
                FROM VIZ_SALES
                GROUP BY
                    DATEPART(dw, date_sale),
                    MONTH(date_sale)
            ) AS SalesPerDayOfMonth
            PIVOT (
                SUM(TotalVenta)
                FOR Mes IN ([1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12])
            ) AS PivotTable
            )
SELECT 
Octubre,
(Noviembre/Octubre) -1 AS [%OCTvsNOV],
Noviembre,
(Diciembre/Noviembre) -1 AS [%NOVvsDIV],
Diciembre
FROM TGNR

(7 rows affected)

Total execution time: 00:00:00.071

Octubre,%OCTvsNOV,Noviembre,%NOVvsDIV,Diciembre
10640.000000,-0.362782,6780.000000,3.367994,29615.000000
16730.000000,0.417812,23720.000000,-0.187817,19265.000000
10815.000000,0.642163,17760.000000,0.398085,24830.000000
10890.000000,1.694674,29345.000000,-0.461238,15810.000000
20065.000000,-0.413158,11775.000000,0.533757,18060.000000
12245.000000,0.347897,16505.000000,-0.071494,15325.000000
14745.000000,-0.355036,9510.000000,3.548370,43255.000000


¿Qué porcentaje de las ventas se ve afectado por aquellas órdenes devueltas y canceladas?

In [214]:
WITH TGEN AS (
    SELECT
        SUM(unit_price * quantity) AS TotalVentas
    FROM VIZ_SALES
),
TGEN_DESC AS (
    SELECT
        CASE description
            WHEN 'Cancelled' THEN 'Canceled/Returned'
            WHEN 'Returned' THEN 'Canceled/Returned'
        ELSE description
        END GroupDescription,
        description,
        SUM(unit_price * quantity) AS Ventas
    FROM VIZ_SALES
    GROUP BY description
)
SELECT 
    TD.GroupDescription,
    TD.description,
    TD.Ventas,
    (TD.Ventas * 1.0 / TG.TotalVentas) AS [%Share],
    (SUM(TD.Ventas) OVER(PARTITION BY GroupDescription) * 1.0 / TG.TotalVentas) AS [%SharebyGroup]
    
FROM TGEN_DESC TD, TGEN TG



(5 rows affected)

Total execution time: 00:00:00.066

GroupDescription,description,Ventas,%Share,%SharebyGroup
Canceled/Returned,Cancelled,78740.000000,0.208480,0.382541
Canceled/Returned,Returned,65740.000000,0.174060,0.382541
Created,Created,74495.000000,0.197241,0.197241
Fulfilled,Fulfilled,85085.000000,0.225280,0.225280
Processed,Processed,73625.000000,0.194937,0.194937


¿Cuántos productos y clientes no han tenido durante el último trimestre del año 2023?

In [4]:
SELECT 
    id_sale
    ,CS.*
FROM Sales AS SL
FULL OUTER JOIN Customer AS CS ON SL.id_customer = CS.id_customer
WHERE id_sale is null

-- LEFT JOIN Products AS PD ON SL.id_product =  PD.id_product 
-- LEFT JOIN orden_status AS OST ON SL.id_status = OST.id_status

(3 rows affected)

Total execution time: 00:00:00.046

id_sale,id_customer,date_creation,customer_name,id_region,id_country
NULL,4,2023-10-18,Emma Davis,2,2
NULL,17,2023-10-22,Michael Turner,2,3
NULL,28,2023-12-16,Dylan Morris,3,1


In [5]:
SELECT 
    id_sale
    ,PD.*
FROM Sales AS SL
FULL OUTER JOIN Products AS PD ON SL.id_product =  PD.id_product
WHERE id_sale is null

(2 rows affected)

Total execution time: 00:00:00.048

id_sale,id_product,product_name,category,unit_price
NULL,7,Zoom Pegasus,Pegasus,135.000000
NULL,14,ACG Terra Gobe,ACG,140.000000
